## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/134bert-base-last/' + '*.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/134bert-base-last/134-BERT_base_unc raw_model_fold_1_last_epoch.h5
/kaggle/input/134bert-base-last/134-BERT_base_unc raw_model_fold_2_last_epoch.h5
/kaggle/input/134bert-base-last/134-BERT_base_unc raw_model_fold_3_last_epoch.h5
/kaggle/input/134bert-base-last/134-BERT_base_unc raw_model_fold_4_last_epoch.h5
/kaggle/input/134bert-base-last/134-BERT_base_unc raw_model_fold_5_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=False)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.942117,0.575713,0.202639,0.462879,0.571804,0.587770,0.688512,0.636413,0.633011,...,0.879290,0.909264,0.546393,0.964134,0.955017,0.829689,0.051386,0.020390,0.917672,0.887092
1,46,0.862944,0.535713,0.005452,0.827119,0.729066,0.923736,0.587242,0.468897,0.024189,...,0.696719,0.957217,0.653621,0.976505,0.983352,0.884443,0.961049,0.055073,0.033872,0.873853
2,70,0.910514,0.630560,0.018678,0.853686,0.877097,0.970517,0.595127,0.456934,0.097174,...,0.848386,0.955404,0.640150,0.977225,0.980475,0.885386,0.095283,0.045944,0.905277,0.923821
3,132,0.886086,0.369577,0.004636,0.684729,0.780529,0.917223,0.557526,0.473843,0.095320,...,0.675202,0.956970,0.669196,0.973967,0.981780,0.900375,0.905228,0.118050,0.479080,0.909150
4,200,0.938855,0.385777,0.027935,0.864383,0.722147,0.858032,0.673945,0.598438,0.092081,...,0.631635,0.929504,0.608750,0.974619,0.970126,0.841511,0.131910,0.112268,0.620461,0.895223


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.882967,0.586194,0.026418,0.752703,0.825186,0.875374,0.580784,0.490858,0.218000,...,0.783650,0.936111,0.661673,0.964108,0.972907,0.867979,0.540129,0.125625,0.487327,0.906009
std,2812.670060,0.055519,0.142307,0.054532,0.132060,0.120378,0.131195,0.057880,0.091399,0.214709,...,0.095645,0.030073,0.050616,0.015831,0.014772,0.048841,0.345207,0.074445,0.312063,0.026502
min,39.000000,0.719596,0.315627,0.001751,0.237470,0.258424,0.218442,0.453498,0.282351,0.003333,...,0.540417,0.775833,0.524954,0.893021,0.892125,0.673806,0.004705,0.003710,0.006534,0.807230
25%,2572.000000,0.846690,0.459433,0.005104,0.673170,0.776921,0.851815,0.538889,0.428317,0.059247,...,0.706308,0.921371,0.627584,0.955612,0.966513,0.841475,0.171968,0.068767,0.184471,0.890060
50%,5093.000000,0.887211,0.580753,0.007705,0.759613,0.839194,0.920631,0.569975,0.465633,0.130089,...,0.786945,0.942498,0.663363,0.967002,0.976808,0.877060,0.636532,0.122963,0.491760,0.909758
75%,7482.000000,0.928789,0.702184,0.018292,0.856048,0.911567,0.953161,0.617651,0.538650,0.326925,...,0.870554,0.956901,0.696518,0.975935,0.982717,0.900896,0.867220,0.167860,0.772766,0.926000
max,9640.000000,0.986194,0.925602,0.496984,0.986708,0.992275,0.988991,0.752500,0.772577,0.859774,...,0.967312,0.984237,0.815664,0.990854,0.993863,0.973604,0.985416,0.519698,0.993832,0.958913
